# Linear and spatial regression
## GGR473 Lab 5 [part 1]

In [ ]:
import statsmodels.api as sm
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm


### Data
You will be using data that represent the count, density, and aggregated characteristics of AirBnB properties within census tracts in Toronto. 

Data have been formatted and prepared for the lab, however, you can explore the data sources here:

- http://insideairbnb.com/get-the-data/
- https://datacentre.chass.utoronto.ca/census/

Census data are from 2021 and AirBnB listings from 2023. The temporal mismatch in the data, as well as the use of aggregated area-level data for analysis highlight key limitations in relation to data quality,  ecological fallacy, and the modifiable areal unit problem (MAUP). Note that these would make strong discussion points for the group project report.

While we will be importing data from a shapefile, consider how this could also be queried from a database (as shown in lab5-0_accessDBdata.ipynb).

In [ ]:
# Import shapefile and store data as geopandas geodataframe
gdf = gpd.read_file("data/airbnbct23.shp")

# Explore the geodataframe by displaying the top ten rows and checking column names and data types
print(gdf.head())
print(gdf.info())

In [ ]:
# Visualize the data's geometries by plotting them
gdf.plot()
plt.title('Spatial plot of geometries')
plt.show()

# Check the distribution of an attribute with a histogram
gdf['incomemed'].plot(kind='hist', bins=20)
plt.title('Histogram of neighbourhood median income')
plt.show()

### Simple linear regression
Linear regression enables the relationship between two or more variables to be modelled. Models assume a linear relationship between the independent variables (explanatory, predictor or input features) and the dependent variable (target outcome). The model estimates the coefficients that best fit the data to assess existing relationships and may be applied predict the target variable.

**Ordinary Least Squares (OLS)** is a commonly method used to estimate the parameters (slope coefficients and intercept) of a linear regression model. It determines the parameters that best fit the observed data by minimizing the sum of the squared differences between the predicted and actual values.

Let's start with a **simple** OLS model that estimates the relationship between two variables: income (independent variable) and AirBnB density (dependent variable). We will use the [statsmodels](https://www.statsmodels.org/dev/generated/statsmodels.regression.linear_model.OLS.html) library.

- incomemed: median income per census tract
- airbnbdens: number of AirBnB properties per square km within census tract

We can see from the histogram plot above that there are a small number of census tracts with an income of zero. As a census tract with an income of zero doesn't make sense, we will remove for the analysis for simplicity. However, you might want to think about how these missing values could be imputed based on a weighted average of neighbouring census tracts.

In [ ]:
gdf = gdf[gdf['incomemed'] > 0]

# Access the income and price fields and set as variables
X = gdf['incomemed'] # independent variable
y = gdf['airbnbdens'] # dependent variable

In [ ]:
# Fit a simple linear regression model
X = sm.add_constant(X)  # Add a constant term for intercept
m1 = sm.OLS(y, X).fit() 

# Print the regression results
print(m1.summary())

A positive co-efficient and significant p-value (<0.05) suggests that an increase in income is associated with an increase in the price of AirBnB properties.

**R-squared** represents the proportion of variance (or spread of values) in y that is explained by X in the model (typically between 0 and 1). A low R-squared of 0.046 suggests that the model does not effectively explain much of the variability observed in the dependent variable and the model is currently not a great fit for the data.

### Multiple regression
Including multiple variables in regression analysis offers several advantages that contribute to enhancing the explanation of relationships and improving model fit.

There are many ways to select additional variables (e.g., by checking for multicolinearity or by using stepwise selection). However, for simplicity we will draw on our prior knowledge and include variables that we expect to influence the dependent variable, airbnb density. 

Let's add population density and household size into our model:

- popndens21: population density per census tract
- hhsizemean: mean household size per census tract


Note we do not have data on all potential influences, such as housing type or amenities. So while we control for confounding by variables included in the model, we do not control for all potential confounders. This unmeasured confounding is termed residual confounding and can bias effect estimates. Another good talking point for the group project!


In [ ]:
# Update X to include multiple independent variables
X = gdf[['incomemed', 'popnden21', 'hhsizemean']]

# Re-run the regression model
X2 = sm.add_constant(X)  # Add a constant term for intercept
m2 = sm.OLS(y, X2).fit() 

print(m2.summary())

Notice what has happened to the r-squared value. Can you explain the effects shown for each independent variable?

### Machine learning
[Scikit-learn](https://scikit-learn.org/stable/supervised_learning.html#supervised-learning) is a great library to use for doing machine learning in Python. Though we will use it for linear regression, the library is worth exploring for its many options for data preparation, exploratory data analysis (EDA), classification, regression, and clustering!

Training sets are used to facilitate model training and test sets are used to assess model performance.
Below, the data are randomly divided into training and test sets using a fixed proportion. **test_size=0.2** specifies that 20% of the data will be allocated to the test set and the remaining 80% will be assigned to the training set.

Splitting data into training and test sets allows for model performance to be evaluated on unseen data. This  helps to  estimate how well the model might perform on new, unseen observations. In this example, this could be for census tracts where there are no data available for AirBnb listings.


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Linear Regression model
m3 = LinearRegression()
m3.fit(X_train, y_train)

# Predict on test set
y_pred_linear = m3.predict(X_test)

# Evaluate model
print("Coefficients:", m3.coef_)
print("Intercept:", m3.intercept_)

m3_mse = mean_squared_error(y_test, y_pred_linear)
print(f"Linear Regression Model 3 MSE: {m3_mse}")
print("R-squared:", r2_score(y_test, y_pred_linear))

The Mean Squared Error (MSE) of a machine learning linear regression model provides insight into the model's predictive performance. A smaller MSE indicates that, on average, the model's predictions are closer to the actual values in the test set.

Adding/removing variables or creating new variables that better explain relationships in the data are simple approaches that can improve model performance.

### Spatial regression
So far we have worked with linear regression models without any consideration for the spatial structure of our data. Intuitively, the price of AirBnBs is likely influenced by spatial factors such as the proximity to amenities and transport links.

To incorporate spatial weights into our model that account for neighbouring data, we can use the spreg module in [PySAL](https://pysal.org/). Spreg implements a standard OLS routine, but is particularly well suited for regressions on spatial data.

Below we construct a spatial weights matrix using Queen contiguity and add it to the model. The Queen contiguity is one approach to define spatial relationships based on shared boundaries (common edges or vertices).

In [ ]:
import libpysal
from libpysal.weights import Queen
from esda.moran import Moran
from spreg import OLS


In [ ]:
X = gdf[['incomemed', 'popnden21', 'hhsizemean']]

# Convert gdf to numpy array (required for spreg)
X_array = X.to_numpy()
y_array = y.to_numpy()

# Construct spatial weights using queen contiguity
w = Queen.from_dataframe(gdf)

# Calculating spatial autocorrelation
moran = Moran(gdf['airbnbdens'], w)
print(f"Moran's I: {moran.I}")

# Run Spatial OLS Model with multiple independent variables
m4 = OLS(y_array, X_array, w=w, name_y='airbnbdens', name_x=['incomemed', 'popnden21', 'hhsizemean'], name_ds='airbnb-gdf', name_w='Queen', spat_diag=True)
#results_spatial_ols = model_spatial_ols.fit()

# Print regression results
print(m4.summary)

Above we tested for global sptial autocorrelation using the Moran's I statistic (values range from -1 to 1). A value of 0.656 indicates a relatively strong positive spatial autocorrelation for AirBnB density. While this suggests neighboring census tracts tend to have similar values for the AirBnB density, the coefficients of the spatial regression model remain largely unchanged. The **spatial effects** of our independent variables may not therefore be strongly associated the dependent variable.



In [ ]:
# calculate MSE manually for Model 2 (multiple OLS model)
residuals = m2.resid
m2_mse = (residuals ** 2).mean()
print(f"Linear Regression Model 2 MSE: {m2_mse}")

m4_mse = mean_squared_error(y, m4.predy)
print(f"Linear Regression Model 4 MSE: {m3_mse}")

It looks like the MSE improves by adding the spatial weight matrix, despite the coefficients and significance remaining the same. This could mean the model's predictive accuracy has been enhanced after accounting for spatial relationships and the addition of spatial information might help capture some of the unexplained variation in the dependent variable that was previously attributed to random error.

Once you have completed the lab, export your notebook by selection File > Print Preview > Save as PD.

Upload your PDF to Part 1 of the lab.